# Improved Belief Model - Colab Training Notebook

This notebook is set up to train the Improved Belief Model on Google Colab with optional GPU acceleration.

What it does:
- Installs dependencies
- Loads your project into Colab (upload zip or mount Drive)
- Parses existing logs into training states
- Trains the improved belief model (with configurable epochs/sample fraction)
- Saves the trained weights back to Drive or local runtime

Notes:
- If using GPU: Runtime → Change runtime type → Hardware accelerator: GPU.
- Ensure your logs exist (e.g., `logs/game28/mcts_games` and/or `logs/improved_games`).


In [ ]:
# Install system and Python dependencies
pip -q install --upgrade pip
pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip -q install tqdm numpy pyyaml


In [ ]:
#@title Mount Google Drive (optional) and set up project
USE_DRIVE = True  #@param {type:"boolean"}
PROJECT_DIR = "/content/28bot"  #@param {type:"string"}

import os, shutil
from pathlib import Path

if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')
  DRIVE_BASE = "/content/drive/MyDrive"
  if not os.path.exists(f"{DRIVE_BASE}/28bot"):
    os.makedirs(f"{DRIVE_BASE}/28bot", exist_ok=True)
  PROJECT_DIR = f"{DRIVE_BASE}/28bot"

print("Project dir:", PROJECT_DIR)
Path(PROJECT_DIR).mkdir(parents=True, exist_ok=True)


In [ ]:
#@title Upload repo zip or clone (choose one)
MODE = "upload_zip"  #@param ["upload_zip", "git_clone"]
GIT_URL = ""  #@param {type:"string"}

import os, zipfile
from google.colab import files

os.chdir(PROJECT_DIR)

if MODE == "upload_zip":
  print("Please upload a zip containing the 28bot_v2 folder and logs/... folders")
  uploaded = files.upload()
  for fn in uploaded:
    if fn.endswith('.zip'):
      with zipfile.ZipFile(fn, 'r') as z:
        z.extractall(PROJECT_DIR)
      print("Unzipped:", fn)
else:
  if not GIT_URL:
    raise ValueError("Provide a GIT_URL or use upload_zip mode")
  # Use shell command via python to avoid lint in this cell
  import subprocess
  subprocess.run(["bash", "-lc", f"git clone {GIT_URL} {PROJECT_DIR}"])

print("Project contents:", os.listdir(PROJECT_DIR)[:20])
